# Ноутбук для cherry picking линии тренда

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
# sys.path

In [ ]:
from market_oracle_lib import hello
from market_oracle_lib.data import t_bank, yfinance, apimoex
from market_oracle_lib.data.data_funcs import (
    create_data_loaders,
    create_data_frames,
    create_data_sets,
    create_base_data_frames,
)
from market_oracle_lib.consts import RU_DEFAULT_SYMBOLS

In [ ]:
import os
from datetime import datetime, timedelta
from dataclasses import dataclass
from help_src.tg_bot.logger import get_logger
from market_oracle_lib.data.t_bank import (
    find_instrument_by_ticker, get_symbol_data
)
from help_src.patterns.src import add_pivot_column, collect_channel, visualize_data, refine_channel
from help_src.patterns.src import PriceChannel
from dataclasses import dataclass
from help_src.gpt_api.perplexity_api import send_request as perp_send_request

from market_oracle_lib.model import LSTMModel
import torch
from market_oracle_lib.data.data_funcs import prepare_data, FEATURE_COLS
from market_oracle_lib.data import data_funcs
from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from help_src.tg_bot.services.tsf_predict_service import make_prod_predict_by_cbt

from help_src.tg_bot.services.advice_service import (
    get_formatted_pattern_advice,
    get_formatted_semantic_advice_with_gpt,
    get_formatted_time_series_forecast_advice
)


In [ ]:
with open("../.secret_env", "r") as f:
    T_TOKEN = f.read().strip().split("=")[1]
print(T_TOKEN)

## Подбираем хорошую картинку

In [ ]:
ticker = "YDEX"

In [ ]:
data_df = get_symbol_data(
    symbol=ticker,
    token=T_TOKEN,
    interval="1d",
    # start_date="2023-01-01",
)


In [ ]:
data_df = data_df[data_df["Date"] >= "2025-01-01"]
data_df

In [ ]:
window_val = 3
data_df = add_pivot_column(data_df, window_val)

candle = len(data_df) - 1
backcandles = 60
window = 3


In [ ]:
price_channel = collect_channel(
    df=data_df,
    candle=candle,
    backcandles=backcandles,
    window=window
)

price_channel = refine_channel(price_channel, should_refine=True)

In [ ]:
visualize_data(
    df=data_df,
    price_ch=price_channel,
    candle=candle,
    backcandles=backcandles,
    window=window,
    do_show=True,
    title=f"Ценовой канал для {ticker}"
)

In [ ]:
data_df
data_df.to_csv(sep='|', index=False)

## As in prod

In [ ]:
advice_text = await get_formatted_pattern_advice(
    ticker, T_TOKEN,
    window_len=90,
    # granularity="1h",
    img_dir="logs/",
    add_oscilators=["MACD", "SMA", "EMA", "RSI"],
    do_refine=True
)

# if advice_text.is_success and advice_text.image_path:
#     await progress_message.delete()
#     await bot.send_photo(
#         chat_id=callback_query.message.chat.id,
#         photo=FSInputFile(advice_text.image_path)
#     )
# else:
#     await progress_message.edit_text(advice_text.error or "Произошла ошибка")


In [ ]:
advice_text


In [ ]:
import shutil

# Удалить директорию и всё её содержимое
shutil.rmtree("logs")